In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
%matplotlib inline

import os, pickle, re, glob, time
import pandas as pd
import numpy as np
np.set_printoptions(precision=2)

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from scipy import stats
from collections import Counter

sns.set_style('ticks')
pd.set_option('precision', 2)
#np.set_printoptions('precision', 2)

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from lib import utils

In [2]:
args = pickle.load(open('arguments.pickle', 'rb'))

# Load reference

In [3]:
import fastaparser
with open(args.reference_fname) as fasta_file:
    parser = fastaparser.Reader(fasta_file)

    seq_ref = []
    for seq in parser:
        seq_ref += [seq.id, seq.sequence_as_string()]

In [4]:
seq_ref_length = len(seq_ref[1])
seq_ref_length

29903

In [8]:
seq_ref_list = list(seq_ref[1])

In [12]:
pickle.dump(seq_ref_list, 
            open(os.path.join(args.preprocess_dir, 'seq_ref_list.pickle'), 'wb'))

/tmp/ipykernel_26796/3733109034.py:1: ResourceWarning: unclosed file <_io.BufferedWriter name='preprocess/seq_ref_list.pickle'>
  pickle.dump(seq_ref_list,


# หาตำแหน่งทั้งหมดใน genome

In [14]:
startstop_df = pd.read_csv(os.path.join(args.preprocess_dir, 'startstop_df.csv'), index_col='id')
snps_df = pd.read_csv(os.path.join(args.preprocess_dir, 'snps_df.csv'))

#เรียง id
id_list = sorted(list(set(startstop_df.index)))
len(id_list)

3406

In [15]:
#จำนวน sample ที่่ไม่่มี snps
len(set(id_list)-(set(list(snps_df['id']))))

17

In [16]:
#สร้าง array ขนาด sample x position จากนั้นใส่่ 1 ในตำแหน่่งที่มี
align_array = np.zeros((len(id_list),seq_ref_length), dtype=str)
snps_max_df = snps_df.copy()

In [17]:
sum_length,snps_max_df,align_array = utils.length_max_snps(align_array,id_list,startstop_df,snps_max_df)

In [18]:
align_array

array([['1', '1', '1', ..., '', '', ''],
       ['', '1', '1', ..., '', '', ''],
       ['', '', '1', ..., '', '', ''],
       ...,
       ['', '', '', ..., '', '', ''],
       ['', '', '1', ..., '', '', ''],
       ['', '', '', ..., '', '', '']], dtype='<U1')

In [19]:
#ตำแหน่่งใหม่่ทั้งหมดที่่ได้
new_position_df = pd.DataFrame(sum_length, columns=['id','length'])
new_position_df.to_csv(os.path.join(args.preprocess_dir, 'new_position_df.csv'), index=False)
new_position_df.head()

,id,length
0,406973,29845
1,407987,29844
2,407988,29843
3,410535,29409
4,410536,29843


In [21]:
snps_max_df.to_csv(os.path.join(args.preprocess_dir, 'snps_max_df.csv'), index=False)

In [22]:
pickle.dump(align_array, open(os.path.join(args.preprocess_dir, 'align_array.pickle'), 'wb'), protocol=4)

/tmp/ipykernel_26796/2752594013.py:1: ResourceWarning: unclosed file <_io.BufferedWriter name='preprocess/align_array.pickle'>
  pickle.dump(align_array, open(os.path.join(args.preprocess_dir, 'align_array.pickle'), 'wb'), protocol=4)


## Create neucleotide array
หา base ใน genome ของแต่ละ sample

In [23]:
#ตำแหน่ง ที่เป็น 1 เป็นตำแหน่งที่มีในสาย genome (เปลี่ยนเป็น ATCG ตาม ref ก่อน)
for i in range(len(id_list)):
    for j in range(seq_ref_length):
        if align_array[i,j]=='1':
            align_array[i,j] = seq_ref_list[j]

In [24]:
sample_id_dict = dict(zip(id_list, range(len(id_list))))

# ใส่ subject ในตำแหน่ง snps
for _,row in snps_max_df.iterrows():
    inx = sample_id_dict[row['id']]
    align_array[inx][row['position']-1] = row['sbjct']

In [25]:
align_array_df = pd.DataFrame(align_array)
align_array_df.index = id_list

In [27]:
pickle.dump(align_array_df, open(os.path.join(args.preprocess_dir, 'align_array_df.pickle'), 'wb'))

/tmp/ipykernel_26796/3360325936.py:1: ResourceWarning: unclosed file <_io.BufferedWriter name='preprocess/align_array_df.pickle'>
  pickle.dump(align_array_df, open(os.path.join(args.preprocess_dir, 'align_array_df.pickle'), 'wb'))
